In [9]:
import os 
import sys
import pickle 
import hyperspy.api as hs
sys.path.append("..")
import tomondt
from tomondt import Operator, Viewer, utils

import numpy as np
import torch 
import sys
import wandb

sys.path.append("..")

from tqdm import tqdm
from time import time

from dipster.solver import Solver
from dipster import alignments, tomo




In [10]:
rebin = True
using =  3
sampleid = 'Cage'

description = ""
atype = 'model'
meta = {}
meta['tiltseries'] = sampleid+'-'+str(using)+'-ts.pkl'
vol = sampleid +"-"+ str(using)+".vmf"

In [11]:
#Load in Data
ts_path = os.path.join(r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\TiltSeries',r"Rod-0-ts.pkl")
v_path = os.path.join(r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Volumes',r"Rod-0.vmf")
ts = pickle.load(open(ts_path, 'rb'))
#ts.data = ts.data[:,:,:,None]


if rebin == True:
    ts.data = alignments.rebin(ts.data, 4)
    ts.pixelsize = 4.0

In [12]:
os.environ["WANDB_DIR"] = "..\wandb"
run = wandb.init(project='DIP-STER-Rod')
#run = wandb.init(settings=wandb.Settings(start_method="thread"))
net =  Solver(ts).cuda().eval(tomondt.load_vmf(v_path),100,4)
net.params.max_steps=30000
net.params.noise_regularizer = 0.00001
net.params.max_steps=250000
net.params.style_size=16
net.params.up_factor = 8
#net.params.style_size=32
#net.params.up_factor = 4
net.params.depth = 4
net.params.gamma = 0.1
#net.params.hidden_dim= 1024
#net.params.batch_size=4
#net.params.lr=0.0005
net.params.epoch_steps = True
meta = net.params.to_dict()
if net.params.eval_type == 'volume':
    meta['volume'] = vol
artifact = wandb.Artifact(run.name, type=atype, description=description, metadata=meta)

net.train()
net.reconstruct('all', os.path.join(os.getcwd(),"..", "Data", run.name+".vmf"))
#for i in range(100):
#    net.params.max_steps+=10000
#    net.train()
#    net.reconstruct('all', os.path.join(os.getcwd(),"..", "Data", str(i)+'-run-'+run.name+".vmf"))
wandb.finish()




  file.seek(int(position))

  obj = file.read(int(dsize))

wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=19 sec). Create a new API with an integer timeout larger than 19, e.g., `api = wandb.Api(timeout=29)` to increase the graphql timeout.
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=19 sec). Create a new API with an integer timeout larger than 19, e.g., `api = wandb.Api(timeout=29)` to increase the graphql timeout.
